# Code for sparse coding of spectrotemporal data
*Nhat Le, Sep 2017*

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import scipy.io.wavfile
import scipy.signal
import glob
from sklearn.decomposition import PCA

%matplotlib inline

In [2]:
def C_L0_norm(x):
    '''L0 norm function for each element of a vector'''
    if x != 0:
        return 0.0
    else:
        return 1.0

def C_L1_norm(x):
    '''L1 norm function for each element of a vector'''
    return abs(x)

def C_L0_norm_deriv(x):
    '''Derivative of C_L0_norm'''
    if x != 0:
        return 0.0
    else:
        return np.inf
    
def C_L1_norm(x):
    '''Derivative of C_L1_norm'''
    if x < 0:
        return -1.0
    elif x > 0:
        return 1.0
    else:
        return np.inf

In [12]:
np.sign([-1, -2, 1, 0])

array([-1, -1,  1,  0])

In [8]:
def T(um, lamb, norm_type):
    '''Threshold function as described'''
    if norm_type == 'L0':
        return max(um, 0.0)
    elif norm_type == 'L1':
        return max(um - lamb, 0.0)
    else:
        raise ValueError('Invalid norm type')

def find_u_dot(tau, u, A, y, lamb):
    s = T_vec(u, lamb, 'L0')
    b = np.dot(A.T, y)
    A_shift = np.dot(A.T, A) - np.identity(A.shape[1])
    return (b - u - np.dot(A_shift, s)) / tau
    
T_vec = np.vectorize(T)

def find_s(tau, A, y, lamb, niter=100):
    # Initialize
    u = np.zeros(A.shape[1], dtype='float')
    for i in range(niter):
        udot = find_u_dot(tau, u, A, y, lamb)
        #print(udot)
        u += udot
        s = T_vec(u, lamb, 'L1')
        error = y - np.dot(A, s)
        energy = 0.5 * np.linalg.norm(error)**2 + lamb * np.linalg.norm(s, ord=1)
        #print('u=', u)
        print('energy=', energy)
    return T_vec(u, lamb, 'L1')

In [ ]:
y-np.dot(A,s)

In [4]:
#b = np.array([1, 1, 3], dtype='float')
A = np.array([[1,3,4, 8], [4,5,6, 2], [0,8,11, 3]], dtype='float')
s = np.array([3,3,4,5], dtype='float')
#u = np.array([5, -4, 3,0], dtype='float')
y = np.array([30, 49, 79], dtype='float')
tau = 1.0
lamb = 0.01

In [9]:
v = find_s(tau, A, y, 3, niter=20)

energy= 461081702.5
energy= 4771.0
energy= 461081702.5
energy= 4771.0
energy= 461081702.5
energy= 4771.0
energy= 461081702.5
energy= 4771.0
energy= 461081702.5
energy= 4771.0
energy= 461081702.5
energy= 4771.0
energy= 461081702.5
energy= 4771.0
energy= 461081702.5
energy= 4771.0
energy= 461081702.5
energy= 4771.0
energy= 461081702.5
energy= 4771.0


In [7]:
v

array([ 0.,  0.,  0.,  0.])

In [ ]:
def learn_step(y, A, s, eta):
    scol = s[:, np.newaxis]
    ycol = y[:, np.newaxis]
    r = ycol - np.dot(A, scol)
    A_new = A - eta * (np.dot(r, scol.T))
    print(np.linalg.norm(ycol - np.dot(A, scol)))
    return A_new

def do_multiple_learn_steps(y, A, s, eta, nsteps=10):
    for i in range(nsteps):
        A = learn_step(y, A, s, eta)
    return A

def find_optimal_A(y, s):
    scol = s[:, np.newaxis]
    ycol = y[:, np.newaxis]
    n = np.dot(scol.T, scol)[0,0]
    print(scol.shape, ycol.shape)
    return np.dot(ycol, 1.0 / n * scol.T)

In [ ]:
find_optimal_A(y, s)

In [ ]:
A_best

In [ ]:
#a1 = np.dot(ycol, scol.T)
a2 = np.dot(scol.T, scol)[0,0]
A_best = np.dot(ycol, 1.0 / a2 * scol.T)
print(np.linalg.norm(ycol - np.dot(A, scol)))
print(np.linalg.norm(ycol - np.dot(A_best, scol)))

In [ ]:
scol = s[:, np.newaxis]
ycol = y[:, np.newaxis]
r = ycol - np.dot(A, scol)
print(np.dot(r,scol.T))

In [ ]:
do_multiple_learn_steps(y, A, s, eta, 100)

In [ ]:
s = find_s(tau, A, y, lamb, niter=100)

In [ ]:
A = np.random.rand(A.shape[0], A.shape[1])

In [ ]:
A

In [ ]:
find_s(tau,A*100,y*1000,lamb)

In [ ]:
eta = 0.1
do_multiple_learn_steps(y, A, s, eta, 20)

In [ ]:
soundfiles = glob.glob('./American-English/*/*.wav')
sound_combined = np.zeros(0)
for file in soundfiles:
    fs, sound = scipy.io.wavfile.read(file)
    sound_combined = np.concatenate((sound_combined, sound))

In [ ]:
f,t,spectrogram = scipy.signal.spectrogram(sound_combined, fs)
plt.figure(figsize=(20,20))
plt.imshow(np.flipud(spectrogram)[:,:400], cmap='viridis');
plt.grid(False)

In [ ]:
# Segment into overlapping segments
segment_len = 20 #samples
segment_step = 5 #samples
segments_lst = []
for t_start in np.arange(0, len(t) - segment_len, segment_step):
    segments_lst.append(spectrogram[:,t_start:(t_start + segment_len)])

In [ ]:
# Do pca on each segment
X = np.zeros((segments_lst[0].shape[0] * segments_lst[0].shape[1], len(segments_lst)))
for idx, segment in enumerate(segments_lst):
    X[:,idx] = segment.ravel()


In [ ]:
# Perform pca with whitening
pca = PCA(n_components=200, whiten=True)
pca.fit(X.T)
X_red = pca.transform(X.T)

In [ ]:
# Visualize principal components
comp = pca.components_
a = comp[3,:].reshape((129, 20))
plt.grid(False)
plt.imshow(np.flipud(a), cmap='jet')

In [ ]:
A = np.random.rand(comp.shape[0], nfeats)


In [ ]:
A.shape

In [ ]:
# Start learning
comp.shape
iters = 1
nfeats = 800
A = np.random.rand(comp.shape[0], nfeats)

for i in range(iters):
    # Pick a random training example
    idx = np.random.randint(comp.shape[1])
    y = comp[:,idx]
    
    # Find s
    s = find_s(tau, A, y, lamb, niter=100)
    print(s.shape)
    
    # Optimize A
    A = find_optimal_A(y, s)
    print(A.shape)
    

In [ ]:
find_s(tau,A,y,lamb,niter=100)

In [ ]:
comp.shape